In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4])


In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-2.8363e-01, -3.5301e-01, -5.3144e-01, -2.7149e-01,  2.4829e-01,
          2.5008e-01, -3.1087e-01,  1.0727e-01,  1.8428e-01, -2.3134e-02,
          6.4098e-02,  4.3587e-02, -2.0620e-01,  4.2702e-01,  5.9174e-01,
         -5.1212e-01, -4.8619e-01, -3.4397e-02,  6.5667e-01, -5.0367e-01],
        [-4.3010e-01, -3.6109e-02, -2.3428e-01, -1.4761e-01,  1.9348e-01,
          1.9856e-01, -1.5816e-01, -8.0384e-05,  1.1630e-01,  2.7243e-01,
          1.5105e-01, -1.2767e-01, -2.1603e-01,  1.7677e-02,  4.0047e-01,
         -3.6484e-01, -7.4619e-01, -1.1578e-02,  6.9853e-01, -4.7067e-01],
        [-4.0806e-01, -3.8190e-01, -3.6194e-01, -2.1179e-01,  2.4113e-01,
          8.9122e-02,  4.1620e-02, -1.9383e-01, -6.2358e-03,  3.3996e-01,
          1.5765e-01, -1.8877e-02, -2.4675e-01,  4.3069e-01,  3.4012e-01,
         -6.4888e-01, -6.0804e-01, -2.1120e-01,  8.1102e-01, -5.2301e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.2483,

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [11]:
logits

tensor([[ 0.3916, -0.0867, -0.1780,  0.2230, -0.0911,  0.0615,  0.0386, -0.0806,
          0.1831,  0.1409],
        [ 0.4088, -0.1195, -0.2184,  0.2255, -0.2310,  0.0437, -0.1313, -0.1850,
          0.1487,  0.0510],
        [ 0.3898, -0.1183, -0.2033,  0.0708, -0.1147, -0.0102,  0.1675, -0.1216,
          0.3786,  0.0454]], grad_fn=<AddmmBackward0>)

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab.size()

torch.Size([3, 10])

In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0344, -0.0328,  0.0176,  ...,  0.0026,  0.0016,  0.0213],
        [ 0.0214, -0.0041, -0.0318,  ..., -0.0341,  0.0211, -0.0049]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0328, -0.0264], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0167,  0.0372,  0.0418,  ...,  0.0102,  0.0151,  0.0425],
        [ 0.0104,  0.0218, -0.0198,  ..., -0.0025, -0.0098, -0.0210]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 